In [2]:
import torch 
from torch import nn

class DetectorPanel(nn.Module):
    def __init__(self, xy, z, zfromhodoscope):
        super().__init__()
        self.xy = xy
        self.z = z
        self.zfromhodoscope = zfromhodoscope

    def globalz(self):
        return self.z+self.zfromhodoscope
    
    def print(self):
        #print(f"\t Panel has xy={self.xy.data}, z={self.z.data+self.globalz.data}")
        print(f"\t Panel has xy={self.xy.data}, relative z={self.z.data}, z from hodoscope={self.zfromhodoscope.data}, global z={self.globalz()}")
 
class Hodoscope(nn.Module):
    def __init__(self, xy, z):
        super().__init__()
        self.xy = nn.parameter.Parameter(data=xy, requires_grad=True)
        self.z = nn.parameter.Parameter(data=z, requires_grad=True)
        self.hodoscopes = []
        
    def generatehodoscopes(self, relzs):
        self.positionList= [nn.parameter.Parameter(data=i, requires_grad=False) for i in relzs]
        for z in self.positionList:
            self.hodoscopes.append(DetectorPanel(self.xy,z, self.z))

    def print(self):
        print(f"Hodoscope has xy={self.xy.data}, z={self.z.data}")
            
    def printhodoscopes(self):
        for panel in self.panels:
            panel.print()
            
            
    def modifyPosition(self, xy, z):
        #self.xy = nn.parameter.Parameter(xy)
        #self.z = nn.parameter.Parameter(z)
        self.xy.data = xy
        self.z.data = z

    

hod = Hodoscope(torch.Tensor([2,3]), torch.Tensor([5]))
positionList=torch.Tensor([0.,0.3,0.9])

hod.generatePanels(positionList)
hod.print()

hod.printPanels()

hod.modifyPosition(torch.Tensor([1,2]), torch.Tensor([8]))

hod.print()
hod.printPanels()

Hodoscope has xy=tensor([2., 3.]), z=tensor([5.])
	 Panel has xy=tensor([2., 3.]), relative z=0.0, z from hodoscope=tensor([5.]), global z=tensor([5.], grad_fn=<AddBackward0>)
	 Panel has xy=tensor([2., 3.]), relative z=0.30000001192092896, z from hodoscope=tensor([5.]), global z=tensor([5.3000], grad_fn=<AddBackward0>)
	 Panel has xy=tensor([2., 3.]), relative z=0.8999999761581421, z from hodoscope=tensor([5.]), global z=tensor([5.9000], grad_fn=<AddBackward0>)
Hodoscope has xy=tensor([1., 2.]), z=tensor([8.])
	 Panel has xy=tensor([1., 2.]), relative z=0.0, z from hodoscope=tensor([8.]), global z=tensor([8.], grad_fn=<AddBackward0>)
	 Panel has xy=tensor([1., 2.]), relative z=0.30000001192092896, z from hodoscope=tensor([8.]), global z=tensor([8.3000], grad_fn=<AddBackward0>)
	 Panel has xy=tensor([1., 2.]), relative z=0.8999999761581421, z from hodoscope=tensor([8.]), global z=tensor([8.9000], grad_fn=<AddBackward0>)


## Layers and hodoscopes


The default *layer* class `PanelDetectorLayer` in TomOpt uses multiple *panels* `DetectorPanel` to record muon position.

`PanelDetectorLayer` inherits from the `AbsDetectorLayer` and **MUST** provide the following **methods**:

 - `forward`: The forward method to propagate the muons. It is a central aspect of `nn.Module`.
 - `get_cost`
 - `conform_detector`
  - `assign_budget`

and the following **features**:

 - `pos`: the position of the layer, either `above` or `bellow`. 
 - `lw`: the length and width of the layer.
 - `z`: the z position of the top of gthe layer. 
 - `size`: the heigth of the layer such that z - size is the bottom of the layer.
 - `device`: torch.device.

In [ ]:
from tomopt.volume.layer import AbsDetectorLayer
from typing import List, Union, Optional
from torch import Tensor

class Hodoscope:
    pass

class HodoscopeDetectorLayer(AbsDetectorLayer):

    def __init__(self, 
                 pos:str, 
                 *,
                 lw:Tensor,
                 z:float,
                 size:float, 
                 hodoscopes: List[Hodoscope],
    ):
        if isinstance(hodoscopes, list):
            hodoscopes = nn.ModuleList(hodoscopes)

        super().__init__(pos=pos, lw=lw, z=z, size=size, device=self.get_device(hodoscopes))
        self.hodoscopes = hodoscopes

        if isinstance(hodoscopes[0], Hodoscope):
            self.type_label = "hodoscope"
            self._n_costs = len(self.hodoscopes)

    @staticmethod
    def get_device(hodoscopes: nn.ModuleList) -> torch.device:
        r"""
        Helper method to ensure that all panels are on the same device, and return that device.
        If not all the panels are on the same device, then an exception will be raised.

        Arguments:
            panels: ModuleLists of either :class:`~tomopt.volume.panel.DetectorPanel` or :class:`~tomopt.volume.heatmap.DetectorHeatMap` objects on device

        Returns:
            Device on which all the panels are.
        """

        device = hodoscopes[0].device
        if len(hodoscopes) > 1:
            for h in hodoscopes[1:]:
                if h.device != device:
                    raise ValueError("All hodoscopes must use the same device, but found multiple devices")
        return device